# Summary and technique selections

We have an episodic task (Which mean we don't need a discount factor different from 1 ), with continuous state space and descrete and limited actions. 
As the objective is the acquire the maximum average reward (return), we need to select a technique that: 

### Value based models : 
    - Allow to learn the Q-values and idealy leads to the Q*-values, the optimal action values, so that we can  infer the optimal policiy that gives good actions for each state, maximizing the total average reward
    - Solves harder problems, but they cannot deal with the continuous action space
    - Explicit exploration
    - Evaluates states and actions
    - Easier to train off-policy
    - 
    - Techniques : 

        - Expected SARSA : Uses the Bellman equation but with expected Q value for S' instead of sampling S' and A'

        - Q-Learning  : Uses the Bellman optimality equation. Alternatives/additional features around Q-learning could be consideredm such as :
            - Experience Replay to fix the problem of sequential correlated data hurting the performance

            - Target Network to fix the problem of policy diverging or oscillating. This will avoid damaging the main network just because we sample a bad set of examples

            - Reward clipping to deal with unstable gradients due to huge different in the Reward magnitude

            - Double Q-learning to deal with statistical issues.
                - When computing Q-learning, we will be calculating the maximum over samples and not the maximum expectation (mean value). The expectation over samples can be > than the maximum expectation, and the Neural Network will still try to learn it and become over optimistic. It will over appreciate to be in this “State” even though this is just because of the statistical error.In this case, the Q-value explodes, becoming larger and larger over time . We can train 2 NN separately and use NN1 to train NN2 and vice-versa.
                

                
            - Dueling DQN which has multiple heads learning the Q-learning, so that samples that are one sample once (duce to epsilon-greedy, for example) can be relevant in at least one of the heads, instead of being forgotten and almost not considered in the policy.

            - DRQN that uses RNN when we need to learn some hidden process of the environment state (specially when it contains sequential information that is relevant, such as an image moving).  We need to train this architecture in a special way: We can sample a sequence of State/Action/Reward that come one after another, but then we will no longer get independent and identically distributed data. Because of that, DRQN sometimes will diverge and sometimes will be better than the DQN. This can also lead to the problem fixed by the Experience replay.


### Policy based models : 
    - Allow to learn a policy directly that learns good actions based on the state
    - Instead of learning action values, we can learn probability related with good actions and select actions based on that. P
    - Policy gradient methods are more generic and good for continuous and descrete action spaces
    - Innate exploration and stochasticity - This does not mean that we can't just increase the exploration factor
    - Techniques :

        - REINFORCE
        - Advantage Actor-Critic : Actor for learning policies and selecting actions and Critic to evaluate the policy
        - Crossentropy methods


### Exploration/Exploitation methods : 

##### Epsilon Greedy 

    -  In an ε-greedy policy, a small percentage (ε) of the time the agent selects a random action, rather than the action with the highest probability. This encourages the agent to try different actions and discover new, potentially more optimal, policies.

    To implement an ε-greedy policy, first, set a small value for ε (e.g. 0.1). Then, at each time step, generate a random number between 0 and 1. If the random number is less than ε, select a random action. If the random number is greater than ε, select the action with the highest probability according to the actor network. 

##### Boltzmann Exploration

    - In this method, the agent selects actions based on a Boltzmann distribution over the action probabilities output by the actor network. The action probabilities are raised to a temperature parameter (τ), and then normalized to obtain a probability distribution. A higher temperature value (τ) encourages more exploration, while a lower temperature value encourages exploitation.
    Pick action proportional to softmax of shifted normalized Q-values. Pi(a|s) = softmax(Q(s,a)/Tau)

##### Ornstein-Uhlenbeck Noise

    - This method adds additive noise to the action selection process, where the noise is generated by a stochastic process that reflects a random walk in the direction of the mean. This can help the agent escape from local optima and encourage exploration in the state space.

##### Count-Based Exploration

    -  This method uses a count-based exploration bonus, where the agent receives a bonus for visiting new states or taking new actions. This can encourage the agent to explore new areas of the state space and discover new policies.

##### Intrinsic Motivation

    - This method adds a intrinsic reward signal for exploration, where the agent receives a reward for exploring new parts of the state space. This can encourage the agent to try new actions and visit new states, even if they do not immediately lead to a higher external reward.

##### Bayesian UCB

    - For guessing what could be the best action in each state to explore/exploit. How do we learn these beliefs of best actions to take in each state? 
            - Learning parametric P(Q), but this can have the problem that for some actions and state, the chosen distribution will not be good. For example, if we choose to learn P(Q) by trying to approximate it to a normal distribution, but it is a Binomial Dist. then we will be going in a wrong way. 
            - Bayesian Neural Network - Mechanism by which you learn a model that predicts not just one value at its output, but a distribution. It does so by having not just one weight at each connection, but distribution and its weight.
        
##### Entropy factor to implement exploration 

    - Adding an entropy factor to the actor loss can encourage exploration in a reinforcement learning agent. The entropy of a probability distribution is a measure of its randomness or uncertainty, so maximizing the entropy of the action probabilities output by the actor network will encourage the agent to select actions randomly rather than relying solely on the highest probability action.

    In practice, this is done by adding a entropy regularization term to the actor loss. The entropy term is proportional to the negative entropy of the action probabilities. The magnitude of the entropy term determines the strength of the exploration.


    The tf.math.log function is used to take the logarithm of the action probabilities in the select_action function. This is because the tf.random.categorical function requires log-probabilities as input, rather than probabilities.

    The entropy regularization term encourages the agent to have a diverse range of action probabilities, and taking the log of the probabilities helps to preserve this diversity during the sampling process. Sampling from the raw probabilities would result in a skewed distribution, where some actions are more likely to be selected than others. By taking the logarithm of the probabilities, you are able to sample from a more uniform distribution, which will encourage the agent to explore different actions.

    In summary, the tf.math.log function is used in the select_action function to ensure that the entropy exploration approach is correctly implemented.

    Using the argmax to select actions based on the raw probabilities output by the actor network will not result in exploration behavior. The argmax function simply selects the action with the highest probability, which may not be diverse and will not encourage the agent to explore different actions.

    To preserve the exploration behavior in the entropy exploration approach, you should sample actions from the action probabilities, as described in my previous answer. The entropy regularization term added to the actor loss will encourage the agent to have a diverse range of action probabilities, and sampling from these probabilities will result in exploration behavior.

    In summary, if you want to maintain exploration behavior in the entropy exploration approach, you should not use the argmax function to select actions. Instead, you should sample actions from the action probabilities.



Note : 
- All of them are off-policy techniques but only Actor-Critic allows to parallelize and speed up the learning process with A3C - Asynchronous  Actor-Critic.
- Value-based methods needs less data to converge to optimal results
- There are situations where Q-learning will not provide the optimal results because it needs to explore a certain amount of time. But to do so, we will need a explicit exploration method such as epsilon-greedy, which is not a best solution if we have a very risky environment such as the Cliff world. For this reason we need to find a way to make the enviroment still explore, but avoid risky states in the enviroment. To make this change in the Q-learning, we need to get rid of the Max operation, which will turn it into the Expected SARSA. So Q-learning would be optimal without exploration, but for certain environment with high risk, it could be a problem. But SARSA get optimal rewards under current exploration strategy. Without a risky enviroment, Q-learning can still operate better than SARSA.
    
- For evaluation, it's common to use the argmax function to select actions, rather than sampling from the action probabilities. This is because the argmax function selects the action with the highest probability, which corresponds to the agent's most confident prediction.
When evaluating the performance of an agent, it's important to get a clear understanding of the agent's performance on a given task. Using the argmax function to select actions allows you to determine the agent's most confident prediction for each state, which can be useful for identifying areas for improvement.
It's important to note that using the argmax function for evaluation can result in different behavior compared to using it in training, where the agent is encouraged to explore different actions through entropy regularization. As a result, it's important to be careful when interpreting the results of an evaluation, and to take into account the different behavior that results from using the argmax function.
In summary, for evaluation, it's common to use the argmax function to select actions, rather than sampling from the action probabilities. This allows you to determine the agent's most confident prediction for each state.

- For hyperparameter tuning, it's common to use the training total reward as the metric to maximize, rather than the evaluation total reward. This is because the training total reward provides a measure of how well the agent is learning to perform the task, while the evaluation total reward provides a measure of how well the agent is able to perform the task.
The training total reward is a good metric to use for hyperparameter tuning because it provides a measure of how well the agent is learning to perform the task over time. By maximizing the training total reward, you can identify the set of hyperparameters that result in the most efficient and effective learning process.
On the other hand, the evaluation total reward provides a measure of how well the agent is able to perform the task after training, and may not reflect the learning process itself. Using the evaluation total reward as a metric for hyperparameter tuning can result in a suboptimal set of hyperparameters, as the focus is on the final performance of the agent, rather than the efficiency and effectiveness of the learning process.
In summary, for hyperparameter tuning, it's common to use the training total reward as the metric to maximize, rather than the evaluation total reward. This provides a measure of how well the agent is learning to perform the task, which is useful for identifying the set of hyperparameters that result in the most efficient and effective learning process.